In [112]:
from pyspark.sql import SparkSession

In [113]:
from pyspark import SparkConf,SparkContext

In [107]:
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [114]:
d = sqlContext.read.format('csv').options(header=True,inferSchema=True).load('titanic.csv')

In [118]:
d.createTempView("Table1")

In [119]:
d.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [120]:
df = sqlContext.sql("select * from table1")

In [121]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [122]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [123]:
my_cols=df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [124]:
my_final_data = my_cols.na.drop()

In [125]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [126]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [127]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [128]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [129]:
from pyspark.ml.classification import LogisticRegression

In [130]:
from pyspark.ml import Pipeline

In [131]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [132]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [133]:
train_data , test_data = my_final_data.randomSplit([0.7,0.3])

In [134]:
fit_model = pipeline.fit(train_data)

In [135]:
results = fit_model.transform(test_data)

In [136]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [137]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [138]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [139]:
AUC = my_eval.evaluate(results)

In [140]:
AUC

0.7982273974634726